# Forecasting using ARIMA type methods 
* Use SACTYPE as clustering group.
* Perform linear regression for each SACTYPE group, compare the significance \& effect sizes of each variable.
* From the case comp website https://ssc.ca/en/case-study/towards-a-clear-understanding-rural-internet-what-statistical-measures-can-be-used-assess:
    * For comparing rural and municipal internet speeds, it may be important to consider the following:

        * Whether the tile is labelled with a population centre or not;
        * SACTYPE - which provides information on the level of municipal influence as defined by Statistics Canada; and/or,
        * Whether a population centre is a small, medium or large (PCCLASS), or its type classification (PCTYPE). For example, it may be interesting to contrast results from small population centres in the rural areas against large population centres.
* Description of the SACTYPEs:
    1. Census subdivision within census metropolitan area
    2. Census subdivision within census agglomeration with at least one census tract
    3. Census subdivision within census agglomeration having no census tracts
    4. Census subdivision outside of census metropolitan area and census agglomeration area having strong metropolitan influence
    5.	Census subdivision outside of census metropolitan area and census agglomeration area having moderate metropolitan influence
    6.	Census subdivision outside of census metropolitan area and census agglomeration area having weak metropolitan influence
    7.	Census subdivision outside of census metropolitan area and census agglomeration area having no metropolitan influence
    8.	Census subdivision within the territories, outside of census agglomeration

In [1]:
from google.colab import drive
drive.mount('/content/drive')

!pip install geopandas rtree &> /dev/null 

Mounted at /content/drive


In [1]:
import geopandas as gpd
import pandas as pd
import numpy as np
import os
import pickle
import matplotlib.pyplot as plt
from importlib import reload

import sys
sys.path.append(os.path.abspath("/content/drive/MyDrive/shared/ssc22-case-comp/sonny_dir/custom_modules"))
import weighted_average_ver3 as WA

## Load the source data
os.chdir('/content/drive/MyDrive/shared/ssc22-case-comp/')

with open('./dataset/can-speed-tiles-with-dist37.p', 'rb') as file:
  data = pickle.load(file)
print(data.info())
data.head(5)

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 2751464 entries, 0 to 2751463
Data columns (total 23 columns):
 #   Column      Dtype   
---  ------      -----   
 0   quadkey     int64   
 1   avg_d_kbps  int64   
 2   avg_u_kbps  int64   
 3   avg_lat_ms  int64   
 4   tests       int64   
 5   devices     int64   
 6   year        int64   
 7   quarter     object  
 8   conn_type   object  
 9   PRUID       int64   
 10  PRNAME      object  
 11  CDUID       int64   
 12  CDNAME      object  
 13  DAUID       int64   
 14  SACTYPE     int64   
 15  DA_POP      float64 
 16  PCUID       float64 
 17  PCNAME      object  
 18  PCTYPE      float64 
 19  PCCLASS     float64 
 20  geometry    geometry
 21  centroid    object  
 22  distance    float64 
dtypes: float64(5), geometry(1), int64(11), object(6)
memory usage: 482.8+ MB
None


,quadkey,avg_d_kbps,avg_u_kbps,avg_lat_ms,tests,devices,year,quarter,conn_type,PRUID,...,DAUID,SACTYPE,DA_POP,PCUID,PCNAME,PCTYPE,PCCLASS,geometry,centroid,distance
0,23331133131332,11910,1408,27,1,1,2019,Q1,fixed,61,...,61010033,8,590.0,NaN,NaN,NaN,NaN,"POLYGON ((4593360.869 4089469.904, 4593533.055...",POINT (4593377.649610395 4089314.513268585),53.062368
1,23331133133011,14969,1554,25,1,1,2019,Q1,fixed,61,...,61010033,8,590.0,NaN,NaN,NaN,NaN,"POLYGON ((4592705.709 4089714.238, 4592877.874...",POINT (4592722.450906813 4089558.8293624604),53.761584
2,32202103303220,5038,1317,54,1,1,2019,Q1,fixed,61,...,61010054,8,330.0,NaN,NaN,NaN,NaN,"POLYGON ((4736491.486 4146142.702, 4736658.777...",POINT (4736511.988045422 4145995.9290841497),120.234330
3,32220031120102,13419,6169,50,4,1,2019,Q1,fixed,61,...,61010045,8,275.0,NaN,NaN,NaN,NaN,"POLYGON ((4642710.684 4070147.116, 4642884.538...",POINT (4642730.427346777 4069993.021852067),0.219597
4,32220031120103,13587,1095,28,2,1,2019,Q1,fixed,61,...,61010045,8,275.0,388.0,Inuvik,4.0,2.0,"POLYGON ((4642884.538 4070012.633, 4643058.404...",POINT (4642904.29490283 4069858.5401834683),0.000000


In [38]:
reload(WA)

group = 'SACTYPE'

groups = sorted(data[group].unique())

w_dist2 = WA.w_avg2(data, group_col = group, to_avg=['avg_d_kbps', 'avg_u_kbps', 'avg_lat_ms', 'distance'], to_w_sum =['DA_POP'], to_sum=['devices'], weight='tests').sort_values(by=['SACTYPE', 'time']).reset_index(drop=True)
w_dist_out = w_dist2.copy()
w_dist_out['avg_d_mbps'] = w_dist_out['avg_d_kbps']/1000
w_dist_out['avg_u_mbps'] = w_dist_out['avg_u_kbps']/1000
w_dist_out = w_dist_out.drop(columns=['avg_d_kbps', 'avg_u_kbps'])
w_dist_out['devices'] = pd.to_numeric(w_dist_out['devices'])
w_dist_out['tests'] = pd.to_numeric(w_dist_out['tests'])

## change the time variables format to datetime
from datetime import datetime 
d_data = w_dist_out.copy()
years = list(range(2019, 2022))
months = ['03-31', '06-30', '09-30', '12-31']
year_months = []
for year in years:
    for month in months:
        year_months.append(str(year) + '-' + month)
year_quarters = sorted(d_data['time'].unique())

d_data['time'] = d_data['time'].replace(dict(zip(year_quarters, year_months)))
d_data['time'] = pd.to_datetime(d_data['time'])
d_data['time'] = pd.to_datetime(d_data['time'], format('%Y-%m'))

## convert some variables to categorical variables
d_data[group] = d_data[group].astype('category')
d_data['conn_type'] = d_data['conn_type'].astype('category')
d_data.info()

with open('./sonny_dir/can-w-avg-by-sactype.p', 'wb') as file:
    pickle.dump(d_data, file)
d_data

,SACTYPE,conn_type,time,avg_d_kbps,avg_u_kbps,avg_lat_ms,distance,DA_POP,devices,tests
0,1,fixed,2019-Q1,96402.408756,32865.598762,16.805888,0.351731,1583.457267,451070,1510416
1,1,mobile,2019-Q1,73752.165199,16263.549267,36.963119,0.335568,1362.392051,43859,66647
2,1,fixed,2019-Q2,98471.502767,35241.659733,17.185076,0.395125,1602.187510,446283,1480005
3,1,mobile,2019-Q2,74922.289406,16551.755023,40.158909,0.334372,1295.200644,44688,72362
4,1,fixed,2019-Q3,103676.645475,37646.263915,15.953372,0.338627,1584.773562,617618,2012544
...,...,...,...,...,...,...,...,...,...,...
187,8,mobile,2021-Q2,26957.631579,5602.894737,51.705263,80.845287,1548.968421,42,95
188,8,fixed,2021-Q3,43560.738739,10991.455856,33.612613,139.817118,925.527928,293,555
189,8,mobile,2021-Q3,32252.436508,9113.571429,58.769841,128.260197,1281.007937,73,126
190,8,fixed,2021-Q4,63557.548747,9994.899721,23.640669,184.994228,758.945682,240,718


In [76]:
## split training set and validation set
val_time = year_months[-1]
train_idx = d_data['time'] != val_time
val_idx = d_data['time'] == val_time
train_set = d_data.loc[train_idx, :].reset_index(drop=True)
val_set = d_data.loc[val_idx, :].reset_index(drop=True)

In [77]:
# create a subset to store true values and forecasts
f_set = val_set[[group, 'conn_type', 'avg_d_mbps']].sort_values(by=[group, 'conn_type']).reset_index(drop=True)

# As we train & make forecasts, create a column for that model and store it.
# for example, 
# f_set['model1'] = preds
# f_set

# create a dataframe to store the errors
mse_df = pd.DataFrame(val_set.loc[:,[group, 'conn_type']])
mse_df

,SACTYPE,conn_type
0,1,fixed
1,1,mobile
2,2,fixed
3,2,mobile
4,3,fixed
5,3,mobile
6,4,fixed
7,4,mobile
8,5,fixed
9,5,mobile


In [143]:
## Linear regression on the entire data
import statsmodels.api as sm

# Prepare data
### one-hot encode the categories
data_onehot = pd.get_dummies(data = d_data, columns = [group, 'conn_type'], drop_first=True) # drop_first gives k-1 dummy variables instead of k

train_x = data_onehot.loc[train_idx, :].drop(columns=['avg_d_mbps'])
train_y = data_onehot.loc[train_idx, 'avg_d_mbps']
val_x = data_onehot.loc[val_idx, :].drop(columns=['avg_d_mbps'])
val_y = data_onehot.loc[val_idx, 'avg_d_mbps']

data_lin = data_onehot.copy()

### Make time a numerical variable
times = sorted(d_data.loc[:,'time'].unique())
time_ints = [*range(1,len(times)+1)]
data_lin.loc[:,'time_int'] = data_lin.loc[:,'time'].replace(to_replace=times, value = time_ints)
data_lin = data_lin.drop(columns = ['time', 'avg_lat_ms', 'avg_u_mbps','tests'])
data_lin = sm.add_constant(data_lin)

### Split train, val data
lin_train_set = data_lin.loc[train_idx, :]
lin_x_train = lin_train_set.drop(columns=['avg_d_mbps'])
lin_y_train = lin_train_set[['avg_d_mbps']]

lin_val_set = data_lin.loc[val_idx, :]
lin_x_valid = lin_val_set.drop(columns=['avg_d_mbps'])
lin_y_valid = lin_val_set.loc[:,'avg_d_mbps']

### Fit linear regression model
# new_x_train = sm.add_constant(lin_x_train)
new_x_train = lin_x_train
OLS_model1 = sm.OLS(np.asarray(lin_y_train), new_x_train ).fit()

### make predictions
# new_x_valid = sm.add_constant(lin_x_valid)
new_x_valid = lin_x_valid
OLS_preds = OLS_model1.predict(new_x_valid).reset_index(drop=True)
f_set['LR'] = OLS_preds

# ## compute errors
mse_df.loc[:,'LR'] = f_set['LR'] - f_set.loc[:, 'avg_d_mbps']
lr_rmse = np.sqrt(np.sum(np.square(mse_df.loc[:,'LR']))/len(f_set['LR']))
print(lr_rmse)

22.828527306221755


/usr/local/lib/python3.7/dist-packages/statsmodels/tsa/tsatools.py:117: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)


In [142]:
OLS_model1.summary2()

<class 'statsmodels.iolib.summary2.Summary'>
"""
                  Results: Ordinary least squares
===================================================================
Model:               OLS              Adj. R-squared:     0.864    
Dependent Variable:  y                AIC:                1386.1140
Date:                2022-04-25 21:03 BIC:                1427.3303
No. Observations:    176              Log-Likelihood:     -680.06  
Df Model:            12               F-statistic:        93.41    
Df Residuals:        163              Prob (F-statistic): 1.62e-66 
R-squared:           0.873            Scale:              143.56   
-------------------------------------------------------------------
                  Coef.   Std.Err.    t    P>|t|   [0.025   0.975] 
-------------------------------------------------------------------
const            100.5848  12.1819  8.2569 0.0000  76.5302 124.6393
distance          -0.0853   0.0583 -1.4630 0.1454  -0.2005   0.0298
DA_POP            -0.0109   0.0078 -1.3869 0.1674  -0.0263   0.0046
devices            0.0000   0.0000  3.7434 0.0003   0.0000   0.0001
SACTYPE_2         -6.9478   6.7931 -1.0228 0.3079 -20.3617   6.4661
SACTYPE_3        -28.2606   7.0441 -4.0119 0.0001 -42.1701 -14.3511
SACTYPE_4        -62.1886   7.6354 -8.1448 0.0000 -77.2656 -47.1115
SACTYPE_5        -63.9796   8.1227 -7.8767 0.0000 -80.0188 -47.9403
SACTYPE_6        -55.3217   8.2401 -6.7137 0.0000 -71.5928 -39.0506
SACTYPE_7        -71.9382   9.9659 -7.2184 0.0000 -91.6171 -52.2593
SACTYPE_8        -60.8139   9.9138 -6.1343 0.0000 -80.3899 -41.2378
conn_type_mobile  -7.9947   2.0336 -3.9312 0.0001 -12.0104  -3.9790
time_int           3.2556   0.2891 11.2614 0.0000   2.6848   3.8265
-------------------------------------------------------------------
Omnibus:               22.393       Durbin-Watson:          2.759  
Prob(Omnibus):         0.000        Jarque-Bera (JB):       27.121 
Skew:                  0.853        Prob(JB):               0.000  
Kurtosis:              3.888        Condition No.:          4137673
===================================================================
* The condition number is large (4e+06). This might indicate
strong multicollinearity or other numerical problems.
"""

In [150]:
import statsmodels.api as sm
## Perform linear regression on each SACTYPE

### Prepare data

# one-hot encode the categories
data_onehot = pd.get_dummies(data = d_data, columns = ['conn_type'], drop_first=True) # drop_first gives k-1 dummy variables instead of k

# train_x = data_onehot.loc[train_idx, :].drop(columns=['avg_d_mbps'])
# train_y = data_onehot.loc[train_idx, 'avg_d_mbps']
# val_x = data_onehot.loc[val_idx, :].drop(columns=['avg_d_mbps'])
# val_y = data_onehot.loc[val_idx, 'avg_d_mbps']

times = sorted(d_data.loc[:,'time'].unique())
time_ints = [*range(1,len(times)+1)]

data_lin = data_onehot.copy()
data_lin.loc[:,'time_int'] = data_lin.loc[:,'time'].replace(to_replace=times, value = time_ints)
data_lin = data_lin.drop(columns = ['time', 'avg_lat_ms', 'avg_u_mbps', 'tests'])

sactypes = data_lin['SACTYPE'].unique()
# sactypes = [1.0, 2.0]

ols_models = {}
ols_preds = []
for s in sactypes:
    print('processing sactype {s}'.format(s=s))
    lin_subset = data_lin.loc[data_lin['SACTYPE'] == s, :]
    # lin_subset = sm.add_constant(lin_subset)
    subset_x = lin_subset.drop(columns=['SACTYPE', 'avg_d_mbps']).reset_index(drop=True)
    subset_y = lin_subset[['avg_d_mbps']].reset_index(drop=True)

    subset_x_train = subset_x.iloc[0:-1]
    subset_y_train = subset_y.iloc[0:-1]

    subset_x_val = subset_x.iloc[[-1]]
    subset_y_val = subset_y.iloc[[-1]]

    new_x_train = subset_x_train
    ols_model = sm.OLS(np.asarray(subset_y_train), new_x_train).fit()
    # ols_model.fit()
    ols_models[s] = ols_model
    print(ols_model.predict(subset_x_val))


processing sactype 1
23    134.053384
dtype: float64
processing sactype 2
23    118.818464
dtype: float64
processing sactype 3
23    107.321923
dtype: float64
processing sactype 4
23    55.789068
dtype: float64
processing sactype 5
23    62.151835
dtype: float64
processing sactype 6
23    79.993492
dtype: float64
processing sactype 7
23    54.822593
dtype: float64
processing sactype 8
23    40.41847
dtype: float64


In [158]:
aa = ols_models[7.0]
OLSresult = (aa.summary2().tables[1])
OLSresult

,Coef.,Std.Err.,t,P>|t|,[0.025,0.975]
distance,0.666610,0.541093,1.231969,0.233812,-0.470185,1.803406
DA_POP,-0.048470,0.050015,-0.969112,0.345332,-0.153548,0.056607
devices,0.002442,0.003794,0.643779,0.527837,-0.005528,0.010413
conn_type_mobile,15.584482,12.173772,1.280169,0.216738,-9.991663,41.160628
time_int,2.067554,0.714086,2.895386,0.009641,0.567315,3.567793
